In [ ]:
import numpy as np
import pandas as pd
import os   
import open3d as o3d

In [ ]:
dir = "../data/trees/labeled_trees/seidel_direct/Trees_upload"
savedir = "../data/trees/labeled_trees/"
samples_list = os.listdir(dir)
samples_list = [(os.path.join(dir, sample), sample.split("_")[0]) for sample in samples_list]

In [ ]:
VAL=1000
def voxelize(points, voxelsize=0.10):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    #print(f"Voxelize the point cloud with a voxel size of {voxelsize}")
    min_bound, max_bound = np.array([-VAL, -VAL, -VAL]), np.array([VAL, VAL, VAL])
    downpcd, _, idx = pcd.voxel_down_sample_and_trace(voxelsize, min_bound, max_bound)
    print(f"Previous size: {points.shape[0]}, new size {np.asarray(downpcd.points).shape[0]}")
    return np.array(downpcd.points)

In [ ]:
savedir = "../data/trees/seidel-trees"

for idx, (sample, cls) in enumerate(samples_list):
    path = os.path.join(savedir, str(idx) + ".npy")
    if os.path.exists(path):
        pass
    pt = np.loadtxt(sample)
    pt = voxelize(pt)
    
    np.save(path, (cls, pt))
    #print(idx, "done")

In [ ]:
samples_list = os.listdir(savedir)
samples_list = [os.path.join(savedir, sample) for sample in samples_list]
cls = []
data = []
for sample in samples_list:
    s = np.load(sample, allow_pickle=True)
    cls += [s[0]]
    data += [s[1]]

In [ ]:
df = pd.DataFrame({"cls": np.array(cls), "coordinates": np.array(data)})
filter = df.cls.value_counts() > 10
df = df[df.cls.isin(filter.index[filter])]

In [ ]:
entries = np.stack((df.cls.unique(), np.arange(len(df.cls.unique()))), axis=1)
dictionary = dict(entries)
df.loc[:, "cls"] = df.cls.replace(dictionary)

In [ ]:
savedir = "../data/trees/seidel-trees2"
for idx, row in df.iterrows():
    path = os.path.join(savedir, str(idx) + ".npy")
    cls, points = row["cls"], row["coordinates"]
    np.save(path, (cls, points))

# further work with already saved trees

In [ ]:
import os 
import numpy as np
import pandas as pd

savedir = "../data/trees/seidel-trees2"
trees = [np.load(os.path.join(savedir, path), allow_pickle=True) for path in os.listdir(savedir)]
df = pd.DataFrame(np.array(trees), columns=["cls", "pc"])
df = df[df.cls !=3]
cls = df.cls.unique()

In [ ]:
shot = 5
eval = 20
sample_numbers = 5
train_idx = np.empty((sample_numbers, len(cls), shot))
eval_idx = np.empty((sample_numbers, len(cls), eval))
for s_id in range(sample_numbers):
    for i, cl in enumerate(cls):
        subset = df[df.cls == cl]
        sample = subset.sample(n=shot + eval)
        train_sample =  sample.index[0:shot]
        eval_sample = sample.index[shot:]
        train_idx[s_id, i] = train_sample
        eval_idx[s_id, i] = eval_sample

In [ ]:
path = "/user/jschnei2/data/few_shot/"
np.save(path + "train_idx.npy", train_idx.astype(int))
np.save(path + "eval_idx.npy", eval_idx.astype(int))